In [1]:
from photutils.segmentation import (SourceCatalog, deblend_sources, detect_sources, detect_threshold)
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
from astropy.visualization import ZScaleInterval
from astropy.table import vstack
from tqdm import tqdm

In [2]:
img_b = fits.getdata('Img_B.fits')
img_v = fits.getdata('Img_V.fits')
hdr_b = fits.getheader('Img_B.fits')
hdr_v = fits.getheader('Img_V.fits')

In [3]:
#Limpieza de imágenes astronómicas, para que no hayan valores NaN o negativos
def limpiar(img):
    img = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)
    img[img < 0] = 0
    return img
#Aplicamos la limpieza a las imágenes de los filtros B y V
img_b = limpiar(img_b)
img_v = limpiar(img_v)

In [4]:
# Configuración 1: agresiva (fuentes pequeñas y brillantes)
parametro_agresiva = dict(nsigma=2, npixels=80, contrast=0.1)

# Configuración 2: suave (estructuras extendidas)
parametro_suave = dict(nsigma=2.5, npixels=400, contrast=0.3)

In [5]:
# Función de detección general
def detectar_fuentes(img, parametro):
    threshold = detect_threshold(img, nsigma=parametro['nsigma'])
    segm = detect_sources(img, threshold, npixels=parametro['npixels'])
    segm_deb = deblend_sources(img, segm, npixels=parametro['npixels'],
                               nlevels=32, contrast=parametro['contrast'])
    return segm_deb


In [ ]:
# Filtro B
deblend_b_agresivo = detectar_fuentes(img_b, parametro_agresiva)
deblend_b_suave = detectar_fuentes(img_b, parametro_suave)

# Filtro V
deblend_v_agresivo = detectar_fuentes(img_v, parametro_agresiva)
deblend_v_suave = detectar_fuentes(img_v, parametro_suave)

Deblending:   0%|          | 0/416 [00:00<?, ?it/s]

Exception in thread Thread-3:
Traceback (most recent call last):
  File "C:\Users\Valem\anaconda3\envs\astro_env\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Valem\anaconda3\envs\astro_env\lib\site-packages\tqdm\_monitor.py", line 84, in run
    instance.refresh(nolock=True)
  File "C:\Users\Valem\anaconda3\envs\astro_env\lib\site-packages\tqdm\std.py", line 1347, in refresh
    self.display()
  File "C:\Users\Valem\anaconda3\envs\astro_env\lib\site-packages\tqdm\notebook.py", line 171, in display
    rtext.value = right
  File "C:\Users\Valem\anaconda3\envs\astro_env\lib\site-packages\traitlets\traitlets.py", line 716, in __set__
    self.set(obj, value)
  File "C:\Users\Valem\anaconda3\envs\astro_env\lib\site-packages\traitlets\traitlets.py", line 706, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "C:\Users\Valem\anaconda3\envs\astro_env\lib\site-packages\traitlets\traitlets.py", line 1513, in _notify_trait
    self.notify_c

Deblending:   0%|          | 0/73 [00:00<?, ?it/s]

Deblending:   0%|          | 0/514 [00:00<?, ?it/s]

Deblending:   0%|          | 0/110 [00:00<?, ?it/s]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Detección agresiva
imab = ax[0].imshow(deblend_b_agresivo, origin='lower')
ax[0].set_title('Filtro B - Detección agresiva')
cbar_ab = plt.colorbar(imab, ax=ax[0], fraction=0.046, pad=0.04)
cbar_ab.set_label('Identificador de fuente')

# Detección suave
imsb = ax[1].imshow(deblend_b_suave, origin='lower')
ax[1].set_title('Filtro B - Detección suave')
cbar_sb = plt.colorbar(imsb, ax=ax[1], fraction=0.046, pad=0.04)
cbar_sb.set_label('Identificador de fuente')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Detección agresiva
imav = ax[0].imshow(deblend_v_agresivo, origin='lower')
ax[0].set_title('Filtro V - Detección agresiva')
cbar_av = plt.colorbar(imav, ax=ax[0], fraction=0.046, pad=0.04)
cbar_av.set_label('Identificador de fuente')

# Detección suave
imsv = ax[1].imshow(deblend_v_suave, origin='lower')
ax[1].set_title('Filtro V - Detección suave')
cbar_sv = plt.colorbar(imsv, ax=ax[1], fraction=0.046, pad=0.04)
cbar_sv.set_label('Identificador de fuente')

plt.tight_layout()
plt.show()

In [23]:
#Generamos los catalogos filtro B
cat_b_agresivo = SourceCatalog(img_b, deblend_b_agresivo)
cat_b_suave = SourceCatalog(img_b, deblend_b_suave)

tabla_b_agresiva = cat_b_agresivo.to_table(['xcentroid', 'ycentroid', 'area', 'segment_flux'])
tabla_b_suave = cat_b_suave.to_table(['xcentroid', 'ycentroid', 'area', 'segment_flux'])

In [24]:
#Generamos los catalogos filtro V
cat_v_agresivo = SourceCatalog(img_v, deblend_v_agresivo)
cat_v_suave = SourceCatalog(img_v, deblend_v_suave)

tabla_v_agresiva = cat_v_agresivo.to_table(['xcentroid', 'ycentroid', 'area', 'segment_flux'])
tabla_v_suave = cat_v_suave.to_table(['xcentroid', 'ycentroid', 'area', 'segment_flux'])

In [36]:
# Flujo real filtro B
tabla_b_agresiva['segment_flux'] /= hdr_b['EXPTIME']
tabla_b_suave['segment_flux'] /= hdr_b['EXPTIME']

In [26]:
# Flujo real foltro V
tabla_v_agresiva['segment_flux'] /= hdr_v['EXPTIME']
tabla_v_suave['segment_flux'] /= hdr_v['EXPTIME']

In [27]:
# Magnitudes calibradas
zp_b = 21.78705359242756
tabla_b_agresiva['magnitude'] = zp_b - 2.5 * np.log10(tabla_b_agresiva['segment_flux'])
tabla_b_suave['magnitude'] = zp_b - 2.5 * np.log10(tabla_b_suave['segment_flux'])

In [28]:
# Magnitudes calibradas
zp_v = 21.276957915057153
tabla_v_agresiva['magnitude'] = zp_v - 2.5 * np.log10(tabla_v_agresiva['segment_flux'])
tabla_v_suave['magnitude'] = zp_v - 2.5 * np.log10(tabla_v_suave['segment_flux'])

In [33]:
print(f"Fuentes detectadas filto B (agresiva): {len(tabla_b_agresiva)}, Fuentes detectadas filto V (agresiva): {len(tabla_v_agresiva)}")
print(f"Fuentes detectadas filtro B (suave): {len(tabla_b_suave)}, Fuentes detectadas filtro V (suave): {len(tabla_v_suave)}")

tabla_combinada_b = vstack([tabla_b_agresiva[:5], tabla_b_suave[:5]],  metadata_conflicts='silent')
tabla_combinada_v = vstack([tabla_v_agresiva[:5], tabla_v_suave[:5]],  metadata_conflicts='silent')


Fuentes detectadas filto B (agresiva): 600, Fuentes detectadas filto V (agresiva): 673
Fuentes detectadas filtro B (suave): 164, Fuentes detectadas filtro V (suave): 272


In [30]:
print(tabla_combinada_b)

    xcentroid          ycentroid      ...    segment_flux        magnitude     
                                      ...                                      
------------------ ------------------ ... ------------------ ------------------
 616.7052941500339  5.336257912316985 ...  635.7840840657552 14.778779462974757
 861.5203417328883 1.4999291014890255 ... 118.20429382324218 16.605470460500378
 1111.274617371991  8.150004539387423 ... 16597.206745211282 11.236966082751445
1741.0462419033304 1.6782472285301806 ...  74.60556303660074 17.105125561747883
1482.0522948590203 53.575383292692266 ...  2427.123672993978 13.324323826988717
 616.4588211398022  5.202603272795101 ...  598.9136317571005 14.843643097204463
1111.2361561652122  8.075193474503406 ... 16509.782071304322 11.242700240865766
1481.0385075459621  53.74725575562212 ...  2264.487764612834 13.399628646269486
1478.6112876988275 140.39311292528004 ...  26434.93164107005 10.731608113210868
2553.6953015943673 111.77671483855076 ..

In [31]:
print(tabla_combinada_v)

    xcentroid          ycentroid      ...    segment_flux        magnitude     
                                      ...                                      
------------------ ------------------ ... ------------------ ------------------
 640.5134255457803  6.780910567173029 ... 2270.5665583292644 12.886622322233404
  888.110230358737 2.9346422049675542 ...  851.9904402414958 13.950871110574848
1134.8732809258622  9.372367788783937 ...  114674.5990357717  8.628289839359644
  675.300492444352 32.745334494198666 ... 220.73699951171875 15.417270077882552
1505.1368789009223  54.71587074813317 ...  13417.14218864441 10.957807859429888
 640.3748878106511  6.612847862568433 ... 2210.5676714579263 12.915698379120583
1134.8230490986373  9.226073460087967 ... 113848.28143895467   8.63614171694609
1504.7950127390866  54.78917908869667 ... 13049.857483673095 10.987943493034685
1056.9076905702882 62.036856016828594 ...  933.0269035339355  13.85222249838148
3535.1982236603008  62.88925423204278 ..